In [1]:
import ethnicolr
from ethnicolr import census_ln, pred_census_ln
import pandas as pd
from genderize import Genderize

2022-05-24 20:41:10.013534: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-24 20:41:10.013569: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Race/Ethnicity groups follow census groups, trained from US 2010 Census data

Genderize is using social media data

Need to install using jupyterhub account - ethnicolr needs python 3.6

In [2]:
RaceEthGroup = {'pctwhite':'white', 'pctblack':'black', 'pctapi':'asian', 'pctaian':'american_indian_alaskan_native', 'pct2prace':'multi_race', 'pcthispanic':'hispanic'}

In [3]:


names = [{'fname':'amber', 'lname': 'smith', 'title':'paper a'},
        {'fname':'yang','lname': 'zhang', 'title':'paper b'},  
         {'fname':'aliayah','lname': 'jackson', 'title':'paper c'}]

df = pd.DataFrame(names)



In [4]:
df

,fname,lname,title
0,amber,smith,paper a
1,yang,zhang,paper b
2,aliayah,jackson,paper c


In [5]:
 df = census_ln(df, 'lname')

In [6]:
df.head()

,fname,lname,title,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
0,amber,smith,paper a,73.35,22.22,0.40,0.85,1.63,1.56
1,yang,zhang,paper b,0.61,0.09,98.16,0.02,0.96,0.16
2,aliayah,jackson,paper c,41.93,53.02,0.31,1.04,2.18,1.53


In [7]:
df = df.rename(columns = RaceEthGroup)


In [8]:
df.head()

,fname,lname,title,white,black,asian,american_indian_alaskan_native,multi_race,hispanic
0,amber,smith,paper a,73.35,22.22,0.40,0.85,1.63,1.56
1,yang,zhang,paper b,0.61,0.09,98.16,0.02,0.96,0.16
2,aliayah,jackson,paper c,41.93,53.02,0.31,1.04,2.18,1.53


## Genderize - calculate gender

disclaimer: only outputs Male or Female

In [9]:
#testing on gender neutral names to see results
Genderize().get(['Alex','Jessie','Taylor'])


[{'name': 'Alex', 'gender': 'male', 'probability': 0.9, 'count': 411319},
 {'name': 'Jessie', 'gender': 'female', 'probability': 0.79, 'count': 7551},
 {'name': 'Taylor', 'gender': 'male', 'probability': 0.72, 'count': 3370}]

In [10]:
## to get values - create dataframe of the results genderize produces of columns

In [11]:
gender_prob = pd.DataFrame(Genderize().get(df.fname))

In [12]:
gender_prob

,name,gender,probability,count
0,amber,female,0.95,5248
1,yang,male,0.64,3869
2,aliayah,female,1.00,3


In [13]:
## create columns with  male probability and female probability

In [14]:
gender_prob.loc[gender_prob['gender']=='female', 'p_female'] = gender_prob['probability']
gender_prob.loc[gender_prob['gender']=='female', 'p_male'] = 1 - gender_prob['probability']

gender_prob.loc[gender_prob['gender']=='male', 'p_female'] = 1 - gender_prob['probability']
gender_prob.loc[gender_prob['gender']=='male', 'p_male'] = gender_prob['probability']

In [15]:
gender_prob

,name,gender,probability,count,p_female,p_male
0,amber,female,0.95,5248,0.95,0.05
1,yang,male,0.64,3869,0.36,0.64
2,aliayah,female,1.00,3,1.00,0.00


merge dataframes together - keep only p_male, p_female

In [16]:
df.merge(gender_prob, left_on = 'fname', right_on = 'name')

,fname,lname,title,white,black,asian,american_indian_alaskan_native,multi_race,hispanic,name,gender,probability,count,p_female,p_male
0,amber,smith,paper a,73.35,22.22,0.40,0.85,1.63,1.56,amber,female,0.95,5248,0.95,0.05
1,yang,zhang,paper b,0.61,0.09,98.16,0.02,0.96,0.16,yang,male,0.64,3869,0.36,0.64
2,aliayah,jackson,paper c,41.93,53.02,0.31,1.04,2.18,1.53,aliayah,female,1.00,3,1.00,0.00
